### __Option Pricing on S&P 500 Daily Risk Control 10% Index__

__The S&P 500 Daily Risk Control 10% Index (SPXT10UE) is part of S&P Dow Jones Risk Control Indices familly and use the below parameters__

| ***Index Name*** | ***Underlying Risk Index*** | ***Risk Control Level*** | ***Maximum Leverage*** | ***Interest Rate*** | ***Volatility Calculation*** | ***Return Frequency for Volatility*** | ***Lag to Rebalancing Date*** | ***Decay Factor (Short-Term)*** | ***Decay Factor (Long-Term)*** | ***Rebalancing Frequency*** | ***Launch Date*** | ***Bloomberg Tickers***                                                           |
|------------------------------------------------|------------------------------------------------------|------------------------|----------------------|-------------------------|----------------------------|--------------------------------------|-----------------------------|-------------------------------|------------------------------|---------------------------|---------------|---------------------------------------------------------------------------------|
| S&P 500 Daily Risk Control 10% Index|S&P 500 Total Return: SPTR (USD) | 10% | 150%                 | SOFR + 0.02963*         | Exponentially weighted    | Daily                                | 2 days                      | 94%                           | 97%                          | Daily                     | 10-Sep-09     | ***Excess Return:*** SPXT10UE (USD)|


__The S&P 500 Daily Risk Control indices are computed using the below methodology__

$$
\text{Risk Control ER Index Value}_t 
= 
\text{RiskControlERIndexValue}_{rb}
\,\times\,
\Biggl[
1
\;+\;
K_{rb}\,\biggl(\frac{\text{UnderlyingIndex}_t}{\text{UnderlyingIndex}_{t-1}} \;-\; 1\biggr)
\;-\;
K_{rb}\,\Bigl(
  \prod_{i=rb+1}^{t}\bigl(1 + \text{InterestRate}_{i-1} \times \frac{D_{i-1,i}}{360}\bigr)
  \;-\; 1
\Bigr)
\Biggr]
$$


$$
K_{rb} 
= 
\min\!\Bigl(\text{Max }K,\;\frac{\text{Target Volatility}}{\text{Realized Volatility}_{rb-d}}\Bigr)
$$


$$
\text{RealizedVolatility}_{S,t} 
  = \sqrt{\frac{252}{n}\,\text{Variance}_{S,t}}
$$

$$
\text{RealizedVolatility}_{L,t}
  = \sqrt{\frac{252}{n}\,\text{Variance}_{L,t}}
$$

$$
\text{Variance}_{S,t}
  = \lambda_S\,\text{Variance}_{S,t-1}
   + \bigl(1 - \lambda_S\bigr)\,\left[
       \ln\!\Bigl(\frac{\text{UnderlyingIndex}_t}{\text{UnderlyingIndex}_{t-n}}\Bigr)
     \right]^2
$$

$$
\text{Variance}_{L,t}
  = \lambda_L\,\text{Variance}_{L,t-1}
   + \bigl(1 - \lambda_L\bigr)\,\left[
       \ln\!\Bigl(\frac{\text{UnderlyingIndex}_t}{\text{UnderlyingIndex}_{t-n}}\Bigr)
     \right]^2
$$

In [13]:
# -------
# IMPORT
# -------
import numpy as np
import pandas as pd
import plotly.express as px
from python_module.pricing_model import BSMModel, SABRModel, HestonHullWhiteModel

pd.options.display.max_rows = 30
pd.options.display.max_columns = 30
pd.options.display.float_format = '{:,.4f}'.format

In [86]:
# -------
# Price vanilla option without stochastic rates
# -------

# Define input parameters for Heston Hull White model
S0 = 100.0        # Initial stock price
v0 = 0.2**2       # Initial variance
r0 = 0.00         # Initial short rate
b0 = 0.01          # Initial state variable for bonds

# Variance process parameters
kappa_v = 1.0     # Mean reversion speed for variance
theta_v = 0.2**2  # Long-term variance
sigma_v = 0.2     # Volatility of variance

# Interest rate process parameters
kappa_r = 10.0     # Mean reversion speed for rates
theta_r = 0.00    # Long-term interest rate
sigma_r = 0.00001  # Volatility of interest rate

# Correlation parameters
rho_sv = -0.5     # Stock-variance correlation
rho_sr = 0.9      # Stock-rate correlation
rho_vr = -0.5     # Variance-rate correlation

# Simulation parameters
T = 1.0           # Time horizon in years
M = 252           # Number of simulations
N = 10000         # Number of time steps
seed = True       # Use seed for reproducibility
seed_value = 44   # Seed value

init_r_var = (0.2**2)/252

lambda_l = 0.97
lambda_s = 0.94

target_vol = 0.1

max_k = 1.5

lag = 2

excess_return = True

S, v, r = HestonHullWhiteModel.compute_montecarlo(
    S0=S0, v0=v0, r0=r0, b0=b0,
    kappa_v=kappa_v, theta_v=theta_v, sigma_v=sigma_v,
    kappa_r=kappa_r, theta_r=theta_r, sigma_r=sigma_r,
    rho_sv=rho_sv, rho_sr=rho_sr, rho_vr=rho_vr,
    T=T, N=N, M=M,
    seed=seed, seed_value=seed_value)

S_df = pd.DataFrame(S)
r_df = pd.DataFrame(r)

log_returns = np.log(S_df / S_df.shift(1)).pow(2).fillna(init_r_var)

alpha_l = 1 - lambda_l
variance_l = log_returns.ewm(alpha=alpha_l, adjust=False).mean()

alpha_s = 1 - lambda_s
variance_s = log_returns.ewm(alpha=alpha_s, adjust=False).mean()

r_vol_l = np.sqrt(variance_l * 252)
r_vol_s = np.sqrt(variance_s * 252)

realized_vol = r_vol_l.combine(r_vol_s, np.maximum)

K = (target_vol / realized_vol.shift(lag)).clip(upper=max_k).bfill()

eq_return = (S_df.pct_change().fillna(0) * K)
risk_free = K * (r_df * (T / M))
excess_return = eq_return - risk_free
vt_price = excess_return.add(1).cumprod() * 100
vt_call_price = (vt_price.iloc[-1].add(-100).clip(lower=0).multiply(np.exp(-pd.DataFrame(r).multiply((T / M)).sum()))).mean()
print('vt_call_price, ', vt_call_price)
print('vt forward', vt_price.iloc[-1].mean())
print(K.mean().mean())

vt_call_price,  4.347238799103224
vt forward 100.70724207171129
0.5377247707399962


In [85]:
# -------
# Price vanilla option without stochastic rates
# -------

# Define input parameters for Heston Hull White model
S0 = 100.0        # Initial stock price
v0 = 0.2**2       # Initial variance
r0 = 0.00         # Initial short rate
b0 = 0.01          # Initial state variable for bonds

# Variance process parameters
kappa_v = 1.0     # Mean reversion speed for variance
theta_v = 0.2**2  # Long-term variance
sigma_v = 0.9     # Volatility of variance

# Interest rate process parameters
kappa_r = 10.0     # Mean reversion speed for rates
theta_r = 0.00    # Long-term interest rate
sigma_r = 0.00001  # Volatility of interest rate

# Correlation parameters
rho_sv = -0.5     # Stock-variance correlation
rho_sr = 0.9      # Stock-rate correlation
rho_vr = -0.5     # Variance-rate correlation

# Simulation parameters
T = 1.0           # Time horizon in years
M = 252           # Number of simulations
N = 10000         # Number of time steps
seed = True       # Use seed for reproducibility
seed_value = 44   # Seed value

init_r_var = (0.2**2)/252

lambda_l = 0.97
lambda_s = 0.94

target_vol = 0.1

max_k = 1.5

lag = 2

excess_return = True

S, v, r = HestonHullWhiteModel.compute_montecarlo(
    S0=S0, v0=v0, r0=r0, b0=b0,
    kappa_v=kappa_v, theta_v=theta_v, sigma_v=sigma_v,
    kappa_r=kappa_r, theta_r=theta_r, sigma_r=sigma_r,
    rho_sv=rho_sv, rho_sr=rho_sr, rho_vr=rho_vr,
    T=T, N=N, M=M,
    seed=seed, seed_value=seed_value)

S_df = pd.DataFrame(S)
r_df = pd.DataFrame(r)

log_returns = np.log(S_df / S_df.shift(1)).pow(2).fillna(init_r_var)

alpha_l = 1 - lambda_l
variance_l = log_returns.ewm(alpha=alpha_l, adjust=False).mean()

alpha_s = 1 - lambda_s
variance_s = log_returns.ewm(alpha=alpha_s, adjust=False).mean()

r_vol_l = np.sqrt(variance_l * 252)
r_vol_s = np.sqrt(variance_s * 252)

realized_vol = r_vol_l.combine(r_vol_s, np.maximum)

K = (target_vol / realized_vol.shift(lag)).clip(upper=max_k).bfill()

eq_return = (S_df.pct_change().fillna(0) * K)
risk_free = K * (r_df * (T / M))
excess_return = eq_return - risk_free
vt_price = excess_return.add(1).cumprod() * 100
vt_call_price = (vt_price.iloc[-1].add(-100).clip(lower=0).multiply(np.exp(-pd.DataFrame(r).multiply((T / M)).sum()))).mean()
print('vt_call_price, ', vt_call_price)
print('vt forward', vt_price.iloc[-1].mean())
print(K.mean().mean())

vt_call_price,  4.071738245857031
vt forward 100.91337237512711
0.79346552869822


In [79]:
# -------
# Price vanilla option without stochastic rates
# -------

# Define input parameters for Heston Hull White model
S0 = 100.0        # Initial stock price
v0 = 0.16**2       # Initial variance
r0 = 0.00         # Initial short rate
b0 = 0.01          # Initial state variable for bonds

# Variance process parameters
kappa_v = 1.0     # Mean reversion speed for variance
theta_v = 0.16**2  # Long-term variance
sigma_v = 0.2     # Volatility of variance

# Interest rate process parameters
kappa_r = 10.0     # Mean reversion speed for rates
theta_r = 0.00    # Long-term interest rate
sigma_r = 0.5  # Volatility of interest rate

# Correlation parameters
rho_sv = -0.5     # Stock-variance correlation
rho_sr = 0.9      # Stock-rate correlation
rho_vr = -0.5     # Variance-rate correlation

# Simulation parameters
T = 1.0           # Time horizon in years
M = 252           # Number of simulations
N = 10000         # Number of time steps
seed = True       # Use seed for reproducibility
seed_value = 44   # Seed value

init_r_var = (0.16**2)/252

lambda_l = 0.97
lambda_s = 0.94

target_vol = 0.1

max_k = 1.5

lag = 2

excess_return = True

S, v, r = HestonHullWhiteModel.compute_montecarlo(
    S0=S0, v0=v0, r0=r0, b0=b0,
    kappa_v=kappa_v, theta_v=theta_v, sigma_v=sigma_v,
    kappa_r=kappa_r, theta_r=theta_r, sigma_r=sigma_r,
    rho_sv=rho_sv, rho_sr=rho_sr, rho_vr=rho_vr,
    T=T, N=N, M=M,
    seed=seed, seed_value=seed_value)

S_df = pd.DataFrame(S)
r_df = pd.DataFrame(r)

log_returns = np.log(S_df / S_df.shift(1)).pow(2).fillna(init_r_var)

alpha_l = 1 - lambda_l
variance_l = log_returns.ewm(alpha=alpha_l, adjust=False).mean()

alpha_s = 1 - lambda_s
variance_s = log_returns.ewm(alpha=alpha_s, adjust=False).mean()

r_vol_l = np.sqrt(variance_l * 252)
r_vol_s = np.sqrt(variance_s * 252)

realized_vol = r_vol_l.combine(r_vol_s, np.maximum)

K = (target_vol / realized_vol.shift(lag)).clip(upper=max_k).bfill()

eq_return = (S_df.pct_change().fillna(0) * K)
risk_free = K * (r_df * (T / M))
excess_return = eq_return - risk_free
vt_price = excess_return.add(1).cumprod() * 100
vt_call_price = (vt_price.iloc[-1].add(-100).clip(lower=0).multiply(np.exp(-pd.DataFrame(r).multiply((T / M)).sum()))).mean()
print('vt_call_price, ', vt_call_price)
print('vt forward', vt_price.iloc[-1].mean())

vt_call_price,  4.205797694561357
vt forward 100.89869723521757


In [75]:
# -------
# Price vanilla option without stochastic rates
# -------

v0 = 0.16**2       # Initial variance
theta_v = 0.16**2  # Long-term variance
sigma_r = 0.5  # Volatility of interest rate

init_r_var = (0.16**2)/252

lambda_l = 0.97
lambda_s = 0.94

target_vol = 0.1

max_k = 1.5

lag = 2

excess_return = True

S, v, r = HestonHullWhiteModel.compute_montecarlo(
    S0=S0, v0=v0, r0=r0, b0=b0,
    kappa_v=kappa_v, theta_v=theta_v, sigma_v=sigma_v,
    kappa_r=kappa_r, theta_r=theta_r, sigma_r=sigma_r,
    rho_sv=rho_sv, rho_sr=rho_sr, rho_vr=rho_vr,
    T=T, N=N, M=M,
    seed=seed, seed_value=seed_value)

S_df = pd.DataFrame(S)
r_df = pd.DataFrame(r)

log_returns = np.log(S_df / S_df.shift(1)).pow(2).fillna(init_r_var)

alpha_l = 1 - lambda_l
variance_l = log_returns.ewm(alpha=alpha_l, adjust=False).mean()

alpha_s = 1 - lambda_s
variance_s = log_returns.ewm(alpha=alpha_s, adjust=False).mean()

r_vol_l = np.sqrt(variance_l * 252)
r_vol_s = np.sqrt(variance_s * 252)

realized_vol = r_vol_l.combine(r_vol_s, np.maximum)

K = (target_vol / realized_vol.shift(lag)).clip(upper=max_k).bfill()

eq_return = (S_df.pct_change().fillna(0) * K)
risk_free = K * (r_df * (T / M))
excess_return = eq_return - risk_free
vt_price = excess_return.add(1).cumprod() * 100
vt_call_price = (vt_price.iloc[-1].add(-100).clip(lower=0).multiply(np.exp(-pd.DataFrame(r).multiply((T / M)).sum()))).mean()
print('vt_call_price, ', vt_call_price)
print('vt forward', vt_price.iloc[-1].mean())

vt_call_price,  3.7989875835043785
vt forward 100.19028773170537


In [70]:
# -------
# Price vanilla option without stochastic rates
# -------

# Define input parameters for Heston Hull White model
S0 = 100.0        # Initial stock price
v0 = 0.2**2       # Initial variance
r0 = 0.00         # Initial short rate
b0 = 0.0          # Initial state variable for bonds

# Variance process parameters
kappa_v = 1.0     # Mean reversion speed for variance
theta_v = 0.2**2  # Long-term variance
sigma_v = 0.9     # Volatility of variance

# Interest rate process parameters
kappa_r = 10.0     # Mean reversion speed for rates
theta_r = 0.00    # Long-term interest rate
sigma_r = 0.00001  # Volatility of interest rate

# Correlation parameters
rho_sv = -0.5     # Stock-variance correlation
rho_sr = 0.9      # Stock-rate correlation
rho_vr = -0.5     # Variance-rate correlation

# Simulation parameters
T = 1.0           # Time horizon in years
M = 252           # Number of simulations
N = 10000         # Number of time steps
seed = True       # Use seed for reproducibility
seed_value = 44   # Seed value

init_r_var = (0.2**2)/252

lambda_l = 0.97
lambda_s = 0.94

target_vol = 0.1

max_k = 1.5

lag = 2

excess_return = True

S, v, r = HestonHullWhiteModel.compute_montecarlo(
    S0=S0, v0=v0, r0=r0, b0=b0,
    kappa_v=kappa_v, theta_v=theta_v, sigma_v=sigma_v,
    kappa_r=kappa_r, theta_r=theta_r, sigma_r=sigma_r,
    rho_sv=rho_sv, rho_sr=rho_sr, rho_vr=rho_vr,
    T=T, N=N, M=M,
    seed=seed, seed_value=seed_value)

S_df = pd.DataFrame(S)
r_df = pd.DataFrame(r)

log_returns = np.log(S_df / S_df.shift(1)).pow(2).fillna(init_r_var)

alpha_l = 1 - lambda_l
variance_l = log_returns.ewm(alpha=alpha_l, adjust=False).mean()

alpha_s = 1 - lambda_s
variance_s = log_returns.ewm(alpha=alpha_s, adjust=False).mean()

r_vol_l = np.sqrt(variance_l * 252)
r_vol_s = np.sqrt(variance_s * 252)

realized_vol = r_vol_l.combine(r_vol_s, np.maximum)

K = (target_vol / realized_vol.shift(lag)).clip(upper=max_k).bfill()

eq_return = (S_df.pct_change().fillna(0) * K)
risk_free = K * (r_df * (T / M))
excess_return = eq_return - risk_free
vt_price = excess_return.add(1).cumprod() * 100
vt_call_price = (vt_price.iloc[-1].add(-100).clip(lower=0).multiply(np.exp(-pd.DataFrame(r).multiply((T / M)).sum()))).mean()
print('vt_call_price, ', vt_call_price)

vt_call_price,  3.5545952428163714


In [16]:
log_returns

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,...,9985,9986,9987,9988,9989,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
0,0.0002,0.0002,0.0002,0.0002,0.0002,0.0002,0.0002,0.0002,0.0002,0.0002,0.0002,0.0002,0.0002,0.0002,0.0002,...,0.0002,0.0002,0.0002,0.0002,0.0002,0.0002,0.0002,0.0002,0.0002,0.0002,0.0002,0.0002,0.0002,0.0002,0.0002
1,0.0003,0.0000,0.0000,0.0002,0.0001,0.0001,0.0000,0.0002,0.0000,0.0000,0.0001,0.0000,0.0001,0.0000,0.0001,...,0.0001,0.0000,0.0002,0.0003,0.0001,0.0000,0.0000,0.0001,0.0011,0.0001,0.0001,0.0001,0.0002,0.0000,0.0000
2,0.0001,0.0004,0.0000,0.0001,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0004,0.0003,0.0000,0.0000,...,0.0004,0.0000,0.0003,0.0000,0.0002,0.0000,0.0001,0.0000,0.0000,0.0000,0.0000,0.0000,0.0004,0.0000,0.0000
3,0.0001,0.0003,0.0001,0.0000,0.0001,0.0001,0.0009,0.0000,0.0000,0.0002,0.0003,0.0002,0.0003,0.0001,0.0001,...,0.0005,0.0000,0.0001,0.0003,0.0002,0.0000,0.0003,0.0000,0.0001,0.0001,0.0000,0.0003,0.0001,0.0007,0.0000
4,0.0000,0.0001,0.0000,0.0000,0.0002,0.0003,0.0001,0.0000,0.0009,0.0006,0.0001,0.0000,0.0001,0.0000,0.0000,...,0.0000,0.0000,0.0001,0.0006,0.0002,0.0000,0.0000,0.0001,0.0000,0.0001,0.0005,0.0002,0.0002,0.0003,0.0001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,0.0000,0.0000,0.0001,0.0002,0.0002,0.0001,0.0005,0.0000,0.0007,0.0004,0.0006,0.0001,0.0004,0.0000,0.0001,...,0.0001,0.0003,0.0000,0.0000,0.0000,0.0001,0.0000,0.0000,0.0000,0.0002,0.0001,0.0001,0.0000,0.0000,0.0000
249,0.0000,0.0002,0.0002,0.0001,0.0000,0.0001,0.0000,0.0001,0.0001,0.0000,0.0001,0.0001,0.0003,0.0000,0.0006,...,0.0001,0.0005,0.0000,0.0000,0.0000,0.0004,0.0001,0.0000,0.0000,0.0008,0.0000,0.0001,0.0001,0.0003,0.0000
250,0.0001,0.0005,0.0001,0.0000,0.0002,0.0003,0.0000,0.0000,0.0001,0.0002,0.0017,0.0001,0.0000,0.0000,0.0003,...,0.0002,0.0000,0.0002,0.0000,0.0000,0.0001,0.0008,0.0001,0.0002,0.0007,0.0000,0.0001,0.0000,0.0000,0.0000
251,0.0000,0.0007,0.0000,0.0001,0.0001,0.0000,0.0001,0.0000,0.0001,0.0001,0.0004,0.0003,0.0003,0.0004,0.0000,...,0.0000,0.0000,0.0000,0.0004,0.0000,0.0001,0.0000,0.0000,0.0000,0.0001,0.0000,0.0000,0.0003,0.0001,0.0001


In [ ]:
# Simulate option price for a given maturity
F = 100
T = 1
r = 0.04

options = {
    0: {"F": F, "K": 90, "T": T, "r": r, "sigma": 0.21, "option_type": "put"},
    1: {"F": F, "K": 100, "T": T, "r": r, "sigma": 0.2, "option_type": "call"},
    2: {"F": F, "K": 110, "T": T, "r": r, "sigma": 0.195, "option_type": "call"}}
for key in options:
    option_price = BSMModel.compute_option_with_forward(**options[key], compute_greeks=False)
    options[key]["option_price"] = option_price
options_df = pd.DataFrame(options).transpose()
display(options_df)

In [ ]:
# Fit SABR model
alpha, rho, nu = SABRModel.solve_parameters(F=F, T=T, strikes=options_df["K"].to_list(), market_vols=options_df["sigma"].to_list())

In [ ]:
# Display SABR implied volatilities
sabr_vols = dict()
for strike in range(50, 151, 5):
    sabr_vol = SABRModel.compute_sigma(F=100, K=strike, T=T, alpha=alpha, rho=rho, nu=nu, beta=1)
    sabr_vols[strike] = sabr_vol
px.line(pd.Series(sabr_vols), height=400, width=700, title="SABR Implied Volatilities")

In [ ]:
# Heston HullWhite model inputs

S0 = 100.0        # initial spot
v0 = 0.2**2       # initial variance
r0 = 0.02         # initial short rate
b0 = 0.0          # growth spread
q0 = 0.0          # dividend yield

kappa_v = 1.5     # mean reversion speed for variance
theta_v = 0.1**2  # long-run variance
sigma_v = 0.3     # vol-of-vol

kappa_r = 0.01     # mean reversion speed for short rate (Hull-White)
theta_r = 0.00    # long-run short rate
sigma_r = 0.1    # short-rate volatility

rho_sv = -0.1     # corr(S, v)
rho_sr = -0.9      # corr(S, r)
rho_vr = 0.1      # corr(v, r)

In [ ]:
T = 1.0          # time horizon in years
N = 100            # number of Monte Carlo paths
M = 252          # number of time steps
seed = True
seed_value = 42


S, v, r = HestonHullWhiteModel.compute_montecarlo(
    S0, v0, r0, b0,
    kappa_v, theta_v, sigma_v,
    kappa_r, theta_r, sigma_r,
    rho_sv, rho_sr, rho_vr,
    T, N, M,
    seed, seed_value
)

In [ ]:
((pd.DataFrame(S).iloc[-1] - 100).clip(lower=0) * np.exp(-(pd.DataFrame(r) * (T/M)).sum())).mean()

In [ ]:
# Heston HullWhite model inputs

S0 = 100.0        # initial spot
v0 = 0.2**2       # initial variance
r0 = 0.02         # initial short rate
b0 = 0.0          # growth spread
q0 = 0.0          # dividend yield

kappa_v = 1.5     # mean reversion speed for variance
theta_v = 0.1**2  # long-run variance
sigma_v = 0.3     # vol-of-vol

kappa_r = 0.3     # mean reversion speed for short rate (Hull-White)
theta_r = 0.00    # long-run short rate
sigma_r = 0.01    # short-rate volatility

rho_sv = -0.7     # corr(S, v)
rho_sr = 0.2      # corr(S, r)
rho_vr = 0.1      # corr(v, r)

In [ ]:
# Generate Past Fixings with stochastic dyamics

T = 1.0          # time horizon in years
N = 1            # number of Monte Carlo paths
M = 252          # number of time steps

seed = True
seed_value = 42

# ---- run simulation ----
risky_asset_fixings, _, short_rate_fixing = HestonHullWhiteModel.compute_montecarlo(
    S0, v0, r0, b0, q0,
    kappa_v, theta_v, sigma_v,
    kappa_r, theta_r, sigma_r,
    rho_sv, rho_sr, rho_vr,
    T, N, M,
    seed, seed_value
)

In [ ]:
risky_asset_fixings_se = pd.DataFrame(risky_asset_fixings)[0]
risky_asset_fixings_se = risky_asset_fixings_se.pct_change().fillna(0).add(1).cumprod() * (S0 / (1 + risky_asset_fixings_se.iloc[-1] / risky_asset_fixings_se.iloc[0] - 1))

In [ ]:
short_rate_fixing_se = pd.DataFrame(short_rate_fixing)[0]
risky_asset_fixings_se = risky_asset_fixings_se.pct_change().fillna(0).add(1).cumprod() * (S0 / (1 + risky_asset_fixings_se.iloc[-1] / risky_asset_fixings_se.iloc[0] - 1))

In [ ]:
risky_asset_fixings_se

In [ ]:
# -------
# INPUTS
# -------
F = 100.0
T = 1.0
alpha = 0.2 
beta = 1.0 
rho = -0.9
nu = 2.5
n_steps = 250
n_paths = 2
seed = True
r = 0.03
g = 0.0020
dt = T / n_steps
vol_target = 0.10
max_leverage = 1.5

In [ ]:
# -------
# Simulate 1Y of risky asset price paths
# -------
index_fwd_process, _ = SABRModel.compute_montecarlo(F=F, T=T, alpha=alpha, beta=beta, rho=rho, nu=nu, n_steps=n_steps, n_paths=1, seed=seed, seed_value=2)
index_returns = (index_fwd_process.pct_change() + r*dt + g*dt).fillna(0)
scaling = F / index_returns.add(1).cumprod().iloc[-1].iloc[0]

In [ ]:
# -------
# Simulate path for pricing
# -------
fwd_process, _ = SABRModel.compute_montecarlo(F=F, T=T, alpha=alpha, beta=beta, rho=rho, nu=nu, n_steps=n_steps, n_paths=n_paths, seed=seed, seed_value=50)
eq_return_process = (fwd_process.pct_change() + r*dt + g*dt).fillna(0)

# Create a DataFrame by repeating index_returns values for each column in eq_return_process
index_returns_df = pd.DataFrame(
    data=np.tile(index_returns.values, (1, len(eq_return_process.columns))), 
    index=index_returns.index, 
    columns=eq_return_process.columns
)

eq_return_process = pd.concat([index_returns_df, eq_return_process], axis=0, ignore_index=True)
eq_return_process = eq_return_process.add(1).cumprod()  * scaling

log_returns_squared = np.log(eq_return_process / eq_return_process.shift(1)).pow(2)

# Short-term variance
variance_S = log_returns_squared.ewm(alpha=0.05, adjust=False).mean()
variance_S = np.sqrt(variance_S * 250)

# Long-term variance
variance_L = log_returns_squared.ewm(alpha=0.01, adjust=False).mean()
variance_L = np.sqrt(variance_L * 250)

realized_variance = variance_S.combine(variance_L, np.maximum)

k = (vol_target / realized_variance).clip(upper=max_leverage).fillna(1.0)
k = k.shift(2).fillna(1.0)
k = k.loc[index_returns_df.index[-1]:].reset_index(drop=True)
Vt_pct_change = k * (g*dt + fwd_process.pct_change())

In [ ]:
Vt_pct_change

In [ ]:
fwd_process.pct_change()

In [ ]:
k